In [ ]:
import os 

In [ ]:
exceptions = {
  "item_ultimate_scepter" : "item_aghanims_scepter_2",
  "item_aghanims_scepter_5" : "item_ultimate_scepter_2",
  "item_boots_of_bearing" : "item_greater_boots_of_bearing_1",
  "item_guardian_greaves" : "item_greater_guardian_greaves",
  #"item_power_treads" : "item_greater_power_treads",
  "item_travel_boots_oaa" : "item_greater_travel_boots",
}
exceptions.keys()

In [ ]:
dirs = [".", "./blinks", "./custom"]
#dirs = ["./blinks"]
for path in dirs:
  items = {}
  for file in os.listdir(path):
    if os.path.isfile(os.path.join(path, file)) and file.endswith(".txt"):
      with open(os.path.join(path, file), "r+") as f:
        primed = False
        for line in f:
          if "item_" in line and line.count('"') == 2 and not "recipe" in line:
            items[file] = line.replace('"', '').strip()

  print(items)
  for file in os.listdir(path):
    if os.path.isfile(os.path.join(path, file)) and file.endswith(".txt"):
      with open(os.path.join(path, file), "r+") as f:
        contents = f.read()
        lines = contents.splitlines()

        item = items[file]
        item_upgrade = None
        if item[-1].isdigit():
          item_upgrade = item[:-1] + str(int(item[-1]) + 1)
        else:
          item_upgrade = item + "_2"

        max_upgrade_level = None
        item_base_level = None
        main_location = None
        backup_location = None
        create_key = True

        for i, line in enumerate(lines):
          line = line.strip()
          if line.startswith('"MaxUpgradeLevel"'):
            max_upgrade_level = int(line.split('"')[3])
          elif line.startswith('"ItemBaseLevel"'):
            item_base_level = int(line.split('"')[3])
            backup_location = i
          elif line.startswith('"UpgradesItems"'):
            main_location = i
            create_key = False

        if ((item_upgrade in items.values() or max_upgrade_level == item_base_level) and max_upgrade_level and item_base_level) or item in exceptions.keys():
          result = ""
          new_line = ""

          if max_upgrade_level == item_base_level:
            new_line = f'    "UpgradesItems"                                       ""'
            result = f'{item} at max level'
          else:
            new_line = f'    "UpgradesItems"                                       "{item_upgrade}"'
            result = f'{item} upgrades into {item_upgrade}'
          if item in exceptions.keys():
            new_line = f'    "UpgradesItems"                                       "{exceptions[item]}"'
            result = f'{item} upgrades into {exceptions[item]}'

          if create_key:
            lines.insert(backup_location + 1, new_line)
            result = result + "; needed to create key"
          else:
            lines[main_location] = new_line


          f.seek(0)
          f.write("\n".join(lines))
          f.truncate()
          print(result)
        else:
          print(f'{item} does not upgrade')
